# Portfolio Management Lecture 05
### Basis of Portfolio Construction & Optimization

This lecture will present a brief introduction to **Portfolio Construction**. It is now clear that financial instruments have 2 common features:

- An expected return
- A certain degree of risk

We also know how to compute measures of return and risk for single assets. But... What happens when we construct portfolios with multiple instruments? And for what reason should we do that?

A single word: **Diversification**.

Remember that **the goal of portfolio management is to target a certain return while lowering risk as much as possible**. To reap the benefits of diversification one needs to invest:

- in multiple instruments
- across different countries and industries

The following parameters are needed to dive deep into Portfolio Construction and Optimization:

- Portfolio Return
- Portfolio Standard Deviation (Risk)

Portfolio return is just the weighted average of the returns of the portfolio components

$$
R_p = \sum_{i=1}^{n} w_i R_i
$$

**Portfolio Standard Deviation** is more complex. In fact, its value is a function of the **correlation** between its components. Imagine having a portfolio with two assets - the variance of the portfolio would have to be computed using the following formula:

$$
\sigma_p^2 = w_1^2 \sigma_1^2 + w_2^2 \sigma_2^2 + 2 w_1 w_2 \sigma_1 \sigma_2 \rho_{12}
$$

We can use some matrix algebra to shorten these formulas!

$$
R_p = w^{T} R
$$

$$
\sigma_p^2 = w^{T} \sum w
$$


Let's create a simple portfolio! 

In [1]:
# import the required libraries 
import numpy as np # Basic operations
import pandas as pd # Data Manipulation
import matplotlib.pyplot as plt # Data Visualization 
from openbb import obb # Market Data Retrieval
obb.user.preferences.output_type = 'dataframe'

In [ ]:
data = obb.equity.price.historical('NVDA,BABA,GOOGL,TSLA',
                                  start_date = '2020-01-01',
                                  end_date = '2025-01-01',
                                  interval = '1M',
                                  provider = 'yfinance')

,open,high,low,close,volume,split_ratio,dividend,symbol
date,,,,,,,,
2020-01-01,216.600006,231.139999,199.500000,206.589996,304096100,0.0,0.0,BABA
2020-01-01,67.420502,75.028999,67.324501,71.639000,673594000,0.0,0.0,GOOGL
2020-01-01,5.968750,6.487500,5.781750,5.910750,6125412000,0.0,0.0,NVDA
2020-01-01,28.299999,43.533333,28.114000,43.371334,6108277500,0.0,0.0,TSLA
2020-02-01,208.669998,226.699997,198.561005,208.000000,329489900,0.0,0.0,BABA


In [6]:
# Data Manipulation 
prices = data.pivot(columns = 'symbol', values = 'close')
prices.head() 

symbol,BABA,GOOGL,NVDA,TSLA
date,,,,
2020-01-01,206.589996,71.639000,5.91075,43.371334
2020-02-01,208.000000,66.962502,6.75175,44.532665
2020-03-01,194.479996,58.097500,6.59000,34.933334
2020-04-01,202.669998,67.334999,7.30700,52.125332
2020-05-01,207.389999,71.676003,8.87550,55.666668


### Portfolio Return

In [ ]:
# Let's compute the returns 
returns = prices.pct_change()[1:]
returns.head() #looks good 

symbol,BABA,GOOGL,NVDA,TSLA
date,,,,
2020-02-01,0.006825,-0.065279,0.142283,0.026776
2020-03-01,-0.065000,-0.132388,-0.023957,-0.215557
2020-04-01,0.042112,0.159000,0.108801,0.492137
2020-05-01,0.023289,0.064469,0.214657,0.067939
2020-06-01,0.040069,-0.010792,0.070109,0.293186


In [ ]:
# Annualized Returns
periods = returns.shape[0]
annualized_returns = (1 + returns).prod() ** (12/periods) - 1
annualized_returns 

symbol
BABA    -0.137092
GOOGL    0.232834
NVDA     0.826242
TSLA     0.563020
dtype: float64

In [12]:
# Let's create a portfolio with equal weights 
assets = len(returns.columns) # 4
weights = np.repeat(1/assets, assets) 
weights 

array([0.25, 0.25, 0.25, 0.25])

In [16]:
portfolio_returns = weights.T @ annualized_returns
portfolio_returns 

np.float64(0.3712512650414984)

### Portfolio Standard Deviation

In [ ]:
cov_matrix = returns.cov()
cov_matrix 

symbol,BABA,GOOGL,NVDA,TSLA
symbol,,,,
BABA,0.016321,0.002650,0.002250,0.004356
GOOGL,0.002650,0.006101,0.005997,0.007186
NVDA,0.002250,0.005997,0.021148,0.013237
TSLA,0.004356,0.007186,0.013237,0.045033


In [ ]:
portfolio_vol = np.sqrt(weights.T@cov_matrix@weights)

np.float64(0.34636120377897034)

A portfolio with equal weights in the four assets would have an annualized return of 0.33.
This was just a simple example of how to compute essential portfolio metrics. **It is obv. difficult to achieve such a return on a consistent basis** since I've picked winners of the AI revolution for this example. 

**In the next lecture we'll dive into Mean-variance Optimization.**